In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import numpy as np

In [ ]:
connection_params = {"user":"shleiferj", 
                     "password":"", 
                     "dbname":"hmda", 
                     "host":"localhost"}

def connect(params=connection_params):
    """
    This function accepts a dictionary of connection parameters that must include:
    - user: the username to be used for the database session
    - password: the user's password
    - dbname: the name of the database for connection
    - host: the host location of the database
    """
    #attempt a connection with the supplied parameters
    try:
        conn = psycopg2.connect(**params)
        print("I'm connected") #print a success message
        return conn.cursor() #return a cursor object
    except psycopg2.Error as e: 
        print("I am unable to connect to the database: ", e) #print a fail message and the error, if any

In [ ]:
def valueCounty(year):
    cur = connect()

    query = f"""
    SELECT property_value, county_code, derived_race
    FROM hmda_public.lar_{year}
    where action_taken='1' and not property_value='Exempt' and not property_value='NA' and total_units = '1';
    """

    cur.execute(query)

    results = cur.fetchall()

    cur.close()

    df = pd.DataFrame(results)
    df.columns = ['value', 'county', 'race']

    return df

In [ ]:
df18 = valueCounty(2018)
df19 = valueCounty(2019)
df20 = valueCounty(2020)
df21 = valueCounty(2021)
df22 = valueCounty(2022)
df23 = valueCounty(2023)

# number of units (single family only)
# types of diffrentiators (enhnaced loan type)
# loan type (conventional, out of conventional) (refinance, chas out refinance, etc.)
# Property types (manufactured home status, number of units, type of build (site, not))

# Specifically look at single family, maybe pull fhfa data (limited liability of a loan)

# os.abspath
# look at spread of price change, look  at top few
# price change has alot to do with little construction (legislation restrictions), diffficult to model

In [ ]:
czconnect = pd.read_csv('/Users/shleiferj/downloads/Citizen_Connect_-_County_data__live__20241105.csv')
# Filter for only commuting and demographic data
commuters = czconnect[czconnect['Category'] == 'Commuting']
demographics = czconnect[czconnect['Category'] == 'Demographics']
del czconnect

resPermits = pd.read_csv('/Users/shleiferj/downloads/Residential_Construction_Permits_by_County_915231567991157423.csv')

# years are separated by columns, so filter for only applicable years
dataRange = ['GEOID']
for year in range(2018, 2023):
    dataRange.append(f'ALL_PERMITS_{year}')
    dataRange.append(f'SINGLE_FAMILY_PERMITS_{year}')
    dataRange.append(f'ALL_MULTIFAMILY_PERMITS_{year}')
    dataRange.append(f'MULTIFAMILY_PERMITS_2_UNITS_{year}')
    dataRange.append(f'MULTIFAMILY_PERMITS_3_4_UNITS_{year}')
    dataRange.append(f'MULTIFAMILY_PERMITS_5_OR_MORE_UNITS_{year}')
# These two are labeled slightly differently, no idea why
dataRange[-2], dataRange[-1] = ['MULTIFAMILY_PERMITS_3_4_UNIT_2022', 'MULTIFAMILY_PERMITS_5_OR_MORE_2022']
resPermits = resPermits[dataRange]
resPermits.index = resPermits['GEOID']


In [ ]:
import functools as ft
from collections import Counter

def num(df, column = 'value'):
    df[column] = pd.to_numeric(df.copy()[column], errors='coerce')
    return df

def count(series):
    return dict(Counter(series))

def size(series):
    return len(series)

def normalCount(series):
    count = dict(Counter(series))
    total = sum(count.values())
    for i in count.keys():
        count[i] = count[i]/total
    return count

df18c = num(df18).groupby('county').agg(
    mean2018 = ('value', 'mean'),
    median2018 = ('value', 'median'),
    race2018 = ('race', count),
    raceNormal2018 = ('race', normalCount),
    count2018 = ('value', size)
)
df19c = num(df19).groupby('county').agg(
    mean2019 = ('value', 'mean'),
    median2019 = ('value', 'median'),
    race2019 = ('race', count),
    raceNormal2019 = ('race', normalCount),
    count2019 = ('value', 'size')
)
df20c = num(df20).groupby('county').agg(
    mean2020 = ('value', 'mean'),
    median2020 = ('value', 'median'),
    race2020 = ('race', count),
    raceNormal2020 = ('race', normalCount),
    count2020 = ('value', 'size')
)
df21c = num(df21).groupby('county').agg(
    mean2021 = ('value', 'mean'),
    median2021 = ('value', 'median'),
    race2021 = ('race', count),
    raceNormal2021 = ('race', normalCount),
    count2021 = ('value', 'size')
)
df22c = num(df22).groupby('county').agg(
    mean2022 = ('value', 'mean'),
    median2022 = ('value', 'median'),
    race2022 = ('race', count),
    raceNormal2022 = ('race', normalCount),
    count2022 = ('value', 'size')
)
df23c = num(df23).groupby('county').agg(
    mean2023 = ('value', 'mean'),
    median2023 = ('value', 'median'),
    race2023 = ('race', count),
    raceNormal2023 = ('race', normalCount),
    count2023 = ('value', 'size')
)
meansMeds = ft.reduce(lambda left, right: pd.merge(left, right, on='county'), [df18c, df19c, df20c, df21c, df22c, df23c])

# Omits countys where not all years have values
# Should juyst be Us/peurto rico

del df18c, df19c, df20c, df21c, df22c, df23c

In [ ]:
meansMeds.to_csv('meanMedsFiltered.csv')

In [ ]:
for year in range(2019, 2024):
    meansMeds[f'medIncrease{year}'] = meansMeds[f'median{year}']/meansMeds[f'median{year-1}']

meansMeds[f'medIncrease18-23'] = meansMeds[f'median2023']/meansMeds[f'median2019']

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/refs/heads/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import plotly.express as px

def yearlyskew(year):
    df = meansMeds.copy()[[f'mean{year}', f'median{year}']]
    df['skew'] = df[f'median{year}']/df[f'mean{year}']
    fig = px.choropleth(df, geojson=counties, locations=df.index, color='skew',
                            color_continuous_scale="Viridis", featureidkey="id",
                                projection="mercator",
                            labels={'value':'Median House Value'}
                            )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    fig.show()

yearlyskew(2019)
yearlyskew(2020)

In [ ]:
def yearlyPriceChange(year):
    df = meansMeds[[f'medIncrease{year}']].copy()
    fig = px.choropleth(df, geojson=counties, locations=df.index, color=f'medIncrease{year}',
                            color_continuous_scale="Viridis", featureidkey="id",
                                projection="mercator",
                            range_color=(0.8, 1.5),
                            labels={'value':'Median House Value'}
                            )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    fig.show()

# yearlyPriceChange(2019)
yearlyPriceChange(2022)
# Ensure each one has enough records for minimal skew (25 central limit theorum, 50 greater caution)

In [ ]:
# Is the increase larger when only looking at those that increased the previous year?
df = meansMeds[['medIncrease2019', 'medIncrease2020']].copy()
print(df['medIncrease2020'].describe())

df = df[df['medIncrease2019'] >= 1.25]
df['medIncrease2020'].describe()

In [ ]:
def maxDemo(series):
    list = []
    for i in series:
        list.append(max(i, key=i.get))
    return list



In [ ]:
# Only show where most prodominant demographic changed
df = meansMeds.copy()[['raceNormal2018', 'raceNormal2023']]
df['demoMax2018'] = maxDemo(df['raceNormal2018'])
df['demoMax2023'] = maxDemo(df['raceNormal2023'])
df = df[[i != j for i,j in zip(maxDemo(meansMeds['raceNormal2018']), maxDemo(meansMeds['raceNormal2023']))]]

fig = px.choropleth(df, geojson=counties, locations=df.index, color='demoMax2018',
                        color_continuous_scale="Viridis", featureidkey="id",
                        projection="mercator",
                        range_color=(0, 2),
                        labels={'value':'Median House Value'}
                        )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

fig = px.choropleth(df, geojson=counties, locations=df.index, color='demoMax2023',
                        color_continuous_scale="Viridis", featureidkey="id",
                        projection="mercator",
                        range_color=(0, 2),
                        labels={'value':'Median House Value'}
                        )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

print(df[['demoMax2018','demoMax2023']].value_counts())

In [ ]:
fig = px.choropleth(resPermits, geojson=counties, locations=resPermits.index, color='SINGLE_FAMILY_PERMITS_2018',
                        color_continuous_scale="Viridis", featureidkey="id",
                        projection="mercator",
                        range_color=(0,5000),
                        labels={'value':'Median House Value'}
                        )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

fig = px.histogram(resPermits, x="SINGLE_FAMILY_PERMITS_2018")
fig.show()
